In [ ]:
import bmll2 as b2
b2.get_file('modules/auxiliary_functions.py')

In [ ]:
import auxiliary_functions as af

import random
import math
import pandas as pd
import numpy as np
from pandas import StringDtype

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import LogFormatterSciNotation

In [ ]:
#b2.get_file('test_data/homo_4_immediate(100).csv')
#impact_data = pd.read_csv('homo_4_immediate(100).csv', parse_dates = ['Date', 'Start time', 'End time'])

b2.get_file('test_data/metaorder_data_homogenous_4.csv')
impact_data = pd.read_csv('metaorder_data_homogenous_4.csv', parse_dates = ['Date', 'Start time', 'End time'])

In [ ]:
ticker              = 'GRT'

In [ ]:
stock_AD_data = impact_data[impact_data['RIC'] == ticker][['RIC', 'Date', '20 AD volatility', '20 AD volume']]
stock_AD_data = stock_AD_data.drop_duplicates(subset = ['Date']).reset_index(drop = True)

In [ ]:
b2.get_file(f'top_100(Volume)/{ticker}.csv')
stock_data = pd.read_csv(f'{ticker}.csv', parse_dates = ['DateTime', 'Date'])
stock_data = stock_data.rename(columns = {'Ticker' : 'RIC'})
stock_data = stock_data.sort_values(['DateTime', 'ExchangeSequenceNo'])

In [ ]:
N                   = 20
trader_distribution = 'homogenous'
alpha               = 2
identifier          = f'{trader_distribution}_{N}'

In [ ]:
%%time

impact = []
for date, day_D in stock_data.groupby('Date', sort = True):
        print(date)
        
        trades = day_D.loc[day_D['Price'] != 0]
    
        N = N
        f = af.trader_participation(N = N, method = trader_distribution, alpha = alpha, f_min = 1, f_max = trades.shape[0], seed = 1)
        c = af.cumulative_probs(f)

        if trades.empty:
            continue

        output = af.orders(N = N, trades = trades, cumulative_probs = c)
        for n in range(N):
            
            trader_n_trades = trades.iloc[output[n], ]
           
            if trader_n_trades.empty:
                continue

            trader_n_metaorders = af.metaorders(trader_n_trades)
            
            if len(trader_n_metaorders) == 0:
                continue

            trader_n_features = af.impact_df(trader_n_metaorders, timing_method = 'immediate', impact_method = 'all')
            trader_n_features['20 AD volatility'] = GFI_AD[GFI_AD['Date'] == date]['20 AD volatility'].iloc[0] 
            
            if not trader_n_features.empty and not trader_n_features.isna().all().all():
                impact.append(trader_n_features)


impact   = pd.concat(impact, ignore_index = True)
exclude_cols = ['RIC', 'Date', 'Start time', 'End time']
numeric_cols = [col for col in impact.columns if col not in exclude_cols]
impact[numeric_cols] = impact[numeric_cols].apply(pd.to_numeric, errors = 'coerce')

impact['Date']          = pd.to_datetime(impact['Date']).dt.normalize()
impact['Start time']    = pd.to_datetime(impact['Start time'], format = 'mixed')
impact['End time']      = pd.to_datetime(impact['End time'], format = 'mixed')
impact['duration(min)'] = (impact['End time'] - impact['Start time']).dt.total_seconds() / 60
durations_GFI               = impact['duration(min)']

impact.to_csv(f'{ticker}_time_independence_{identifier}.csv', index = False)
b2.put_file(f'{ticker}_time_independence_{identifier}.csv', 'test_data')
